In [7]:
from model import run_exp
from model import default_params as params
import utils as utl
from brian2 import Hz

config = {
    'path_res'  : './results/example',                              # directory to store results
    'path_comp' : './2023_03_23_completeness_630_final.csv',        # completness data
    'path_con'  : './2023_03_23_connectivity_630_final.parquet',    # connectivity data
    'n_proc'    : -1,                                               # number of CPU cores (-1: use all)
}

# Introduction
## Underlying connectivity data
The connectivity of the fly brain is stored in the folowing files:
- neurons present: `config['path_comp']`
- connectivity between neurons: `config['path_con]`

## Model parameters
The equation and constants for the leaky integrate and fire model are defined 
in the dictionary `default_params` in the beginning of the file `model.py`:

```
default_params = {
    # trials
    't_run'     : 1000 * ms,              # duration of trial
    'n_run'     : 30,                     # number of runs

    'v_0'       : -52 * mV,               # resting potential
    'v_rst'     : -52 * mV,               # reset potential after spike
    [...]
```
We can also change v
alues and pass the modified dictionary to the model (see Experiment 1).

## Example
Here, we want to stimulate some sugar-sensing neurons in the right hemisphere.
The neurons of interest are defined via their flywire IDs:

In [8]:
neu_sugar = [
    720575940624963786,
    720575940630233916,
    720575940637568838,
    720575940638202345,
    720575940617000768,
    720575940630797113,
    720575940632889389,
    720575940621754367,
    720575940621502051,
    720575940640649691,
    720575940639332736,
    720575940616885538,
    720575940639198653,
    720575940620900446,
    720575940617937543,
    720575940632425919,
    720575940633143833,
    720575940612670570,
    720575940628853239,
    720575940629176663,
    720575940611875570,
]

For an easier identification, we define also a mapping from the flywire IDs to custom 
names. The above neurons are calles `sugar_1`, `sugar_2` etc:

In [9]:
flyid2name = { f: 'sugar_{}'.format(i+1) for i, f in enumerate(neu_sugar) }
flyid2name

{720575940624963786: 'sugar_1',
 720575940630233916: 'sugar_2',
 720575940637568838: 'sugar_3',
 720575940638202345: 'sugar_4',
 720575940617000768: 'sugar_5',
 720575940630797113: 'sugar_6',
 720575940632889389: 'sugar_7',
 720575940621754367: 'sugar_8',
 720575940621502051: 'sugar_9',
 720575940640649691: 'sugar_10',
 720575940639332736: 'sugar_11',
 720575940616885538: 'sugar_12',
 720575940639198653: 'sugar_13',
 720575940620900446: 'sugar_14',
 720575940617937543: 'sugar_15',
 720575940632425919: 'sugar_16',
 720575940633143833: 'sugar_17',
 720575940612670570: 'sugar_18',
 720575940628853239: 'sugar_19',
 720575940629176663: 'sugar_20',
 720575940611875570: 'sugar_21'}

To run a simulation exciting these nerons we have to call `run_exp` supplying the following:
- unique name for the simulation: `exp_name`
- a list of neurons we want to stimulate: `neu_sugar`
- the connectivity data: `config['path_comp']` and `config['path_con]`
- path to store the output: `config['path_res']`
- number of CPU cores use: `config['n_procs]`

In [10]:
# this cell is optional, because the simulations have already been performed
run_exp(exp_name='sugarR', neu_exc=neu_sugar, **config)

>>> Experiment:     sugarR
    Output file:    results\example\sugarR.parquet
    Exited neurons: 21
    Elapsed time:   150 s


The `.parquet` file created during a simulation contains all spikes events of all neurons in the model.
We load the data again from disk by passing a list of result files to the `utl.load_exps` function.

We can see from the size of the dataframe
that more than 400 000 spikes were generated by activating the sugar neurons (30 trials, 1 s each).

In [5]:
df_spike = utl.load_exps([ './results/sugarR.parquet' ])
df_spike

,t,trial,flywire_id,exp_name
0,0.1397,0,720575940605513649,sugarR
1,0.4950,0,720575940605513649,sugarR
2,0.6235,0,720575940605513649,sugarR
3,0.7228,0,720575940605513649,sugarR
4,0.8168,0,720575940605513649,sugarR
...,...,...,...,...
410928,0.8900,29,720575940660229505,sugarR
410929,0.9118,29,720575940660229505,sugarR
410930,0.9423,29,720575940660229505,sugarR
410931,0.9617,29,720575940660229505,sugarR


The spike times can be converted to spike rates [Hz] via `utl.get_rate`, which requires the duration of each trial.
`utl.get_rate` returns `pandas.DataFrame` objects:
1. spike rate for each neuron (rows) in each experiment (column): `df_rate`
2. standard deviation of rate across trials: `df_rate_std`

For convenience, we can optionally pass the `flyid2name` dictionary to `utl.get_rate` in order to convert flywire IDs into
meaningful names.

We can see that only about 400 neurons show activity during the simulations.

In [6]:
# load spike rate and standard deviation
df_rate, df_rate_std = utl.get_rate(df_spike, duration=params['t_run'], flyid2name=flyid2name)
# sort by spike rate
df_rate.sort_values('sugarR', ascending=False)

exp_name,name,sugarR
flyid,,
720575940639198653,sugar_13,152.733333
720575940637568838,sugar_3,151.300000
720575940629176663,sugar_20,149.500000
720575940617000768,sugar_5,149.433333
720575940624963786,sugar_1,148.900000
...,...,...
720575940644666148,,1.000000
720575940630403031,,1.000000
720575940629667639,,1.000000


# Experiment 1

We want to change the frequency of the stimulation of the sugar neurons.
To do so we mofify the value for `r_poi` in the `default_params` dictionary and pass the altered dictionary to the `run_exp` function.

Note: Since physical quantities in `brian2` have to have the correct unit, we also need the `brian2.Hz` object 
to define a frequency.

In [ ]:
# this cell is optional, because the simulations have already been performed

freqs =  [20, 40, 60, 80, 100]

for f in freqs:

    params['r_poi'] = f * Hz
    
    run_exp(exp_name='sugarR_{}Hz'.format(f), neu_exc=neu_sugar, params=params, **config)

We load the results via the `utl.load_exps` function and convert the spike events to rates with `utl.get_rate`

In [9]:
ps = [
    './results/sugarR_20Hz.parquet',
    './results/sugarR_40Hz.parquet',
    './results/sugarR_60Hz.parquet',
    './results/sugarR_80Hz.parquet',
    './results/sugarR_100Hz.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, duration=params['t_run'], flyid2name=flyid2name)
df_rate.sort_values('sugarR_100Hz', ascending=False, inplace=True)
df_rate

exp_name,name,sugarR_100Hz,sugarR_20Hz,sugarR_40Hz,sugarR_60Hz,sugarR_80Hz
flyid,,,,,,
720575940622695448,,114.700000,34.133333,65.700000,86.600000,102.533333
720575940632889389,sugar_7,103.633333,19.966667,39.533333,60.633333,76.533333
720575940629888530,,101.633333,30.400000,56.733333,76.033333,90.966667
720575940637568838,sugar_3,101.500000,19.700000,40.733333,58.900000,77.500000
720575940621502051,sugar_9,100.900000,20.033333,39.000000,59.533333,80.433333
...,...,...,...,...,...,...
720575940630233404,,NaN,NaN,1.000000,NaN,NaN
720575940630461404,,NaN,NaN,NaN,1.000000,NaN
720575940630548751,,NaN,NaN,NaN,NaN,1.000000


# Experiment 2
We are interested in the 100 most active neurons:

In [10]:
id_top100 = df_rate.sort_values('sugarR_100Hz', ascending=False).index[:100]

In [ ]:
# this cell is optional, because the simulations have already been performed

params['r_poi'] = 200 * Hz

for i in id_top100:
    run_exp(exp_name=str(i), neu_exc=[ i ], params=params, **config)

In [11]:
ps = [ './results/{}.parquet'.format(i) for i in id_top100 ]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, duration=params['t_run'])
df_rate

exp_name,720575940605301438,720575940606866377,720575940607272649,720575940610054980,720575940611875570,720575940612064113,720575940612423922,720575940612611301,720575940612670570,720575940612692633,...,720575940639332736,720575940640649691,720575940641366517,720575940642428045,720575940643867296,720575940645521262,720575940652580086,720575940655014049,720575940660219265,720575940660223873
flyid,,,,,,,,,,,,,,,,,,,,,
720575940604735660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
720575940604737708,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,1.615385,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.500000,NaN,6.800000
720575940605161388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720575940605301438,197.133333,NaN,94.366667,NaN,NaN,NaN,NaN,61.333333,1.0,2.578947,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,35.766667,NaN,28.466667
720575940605494560,NaN,NaN,4.076923,NaN,NaN,NaN,NaN,2.533333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.875000,NaN,1.230769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720575940660224641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720575940660229505,NaN,NaN,3.862069,NaN,NaN,NaN,NaN,72.666667,NaN,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.633333,NaN,36.066667
720575940660233601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.533333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
id_mn9 = 720575940660219265
df_rate.loc[ id_mn9, df_rate.loc[id_mn9, :] > 0 ]

exp_name
720575940607272649     89.800000
720575940612611301     56.466667
720575940612670570      1.307692
720575940612692633      5.892857
720575940614763666     10.966667
720575940615671106     74.433333
720575940617000768      1.000000
720575940617593233     45.966667
720575940617937543      7.633333
720575940619877396     46.433333
720575940620874757     64.900000
720575940620900446      1.923077
720575940623210445     45.266667
720575940623211725    134.233333
720575940624387848     55.000000
720575940624963786      1.000000
720575940625102692     39.933333
720575940625175054      5.576923
720575940629888530      3.655172
720575940630579574     65.900000
720575940631959097      1.333333
720575940631997032     58.500000
720575940632252743     85.400000
720575940632425919      1.384615
720575940633143833      5.466667
720575940638103349     45.733333
720575940639198653      8.466667
720575940642428045      1.000000
720575940655014049     63.033333
720575940660219265    195.333333
7

# Experiement 3
## Run simulation

In [ ]:
params['r_poi'] = 60 * Hz

for i in id_top100:
    run_exp(exp_name='sugarR-{}'.format(i), neu_exc=neu_sugar, neu_slnc=[ i ], name2flyid=name2flyid, params=params, **config)

In [14]:
ps = [ './results/sugarR-{}.parquet'.format(i) for i in id_top100 ]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, duration=params['t_run'])
df_rate.loc[id_mn9, :].sort_values(ascending=True)

exp_name
sugarR-720575940607272649    18.333333
sugarR-720575940639198653    23.400000
sugarR-720575940612611301    24.066667
sugarR-720575940623211725    25.633333
sugarR-720575940624387848    25.800000
                               ...    
sugarR-720575940652580086    44.366667
sugarR-720575940622695448    47.100000
sugarR-720575940639285949    51.666667
sugarR-720575940612906518    54.200000
sugarR-720575940615041430    71.100000
Name: 720575940660219265, Length: 100, dtype: float64